# Run Training

In [2]:
from datetime import datetime
print("Run at:", datetime.now().strftime("%d.%m.%Y %H:%M:%S"))

Run at: 12.04.2024 13:46:29


In [3]:
# Enable interactive plot
#@formatter:off
%load_ext autoreload
%autoreload 2
#@formatter:on

import torch
import importlib
from run import path_resolution

import visualkeras
# you might need to limit this to version 0.0.1
from pytorch2keras import pytorch_to_keras 
# you might need to remove the optimizer in the import here, as we're not going to optimize anything that shouldn't be a problem

/home/yale1/miniconda3/envs/smart-cities/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-12 13:46:32.862008: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-12 13:46:32.913711: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 13:46:33.823630: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
def load(lib, params, data):
    # load model
    NNModule = importlib.import_module(f"models.{lib}")
    hyper_params = getattr(NNModule, params)
    model = NNModule.NeuralNetwork(model_params=hyper_params['model_params'],
                        optimizer=hyper_params['optimizer'],
                        loss_function=hyper_params['loss_function'],
                        optimizer_params=hyper_params['optimizer_params'],
                        scheduler_params=hyper_params['scheduler_params'])
    
    # construct data module
    DLModule = getattr(importlib.import_module(f"datasets.{data}"), data)
    data_module = DLModule(data_dir=path_resolution(), 
        n_jobs=1, 
        debug=True, 
        **hyper_params['data_params'])
    
    return model, data_module.data_shape

# Plotting

In [5]:
models = [
    dict(
        lib = "OwnBaselineCNN",
        params = "hp_default",
        data = "RSO_LModule"
    ),
    dict(
        lib = "CNNModel_LiChan2014",
        params = "hp_default",
        data = "RSO_LModule"
    ),
    dict(
        lib = "CNNModel_LiChan2014_AvgPool",
        params = "hp_default",
        data = "RSO_LModule"
    ),
    dict(
        lib = "ResNet50_SunShangetAl",
        params = "hp_default",
        data = "RSO_LModule"
    ),
    dict(
        lib = "DenseNet",
        params = "hp_default",
        data = "RSO_LModule"
    ),
]

In [7]:
model, shape = load(**models[0])

2024-04-12 13:47:06,764 - torch.distributed.nn.jit.instantiator - INFO - Created a temporary directory at /tmp/tmp0wwf4g1e
2024-04-12 13:47:06,769 - torch.distributed.nn.jit.instantiator - INFO - Writing /tmp/tmp0wwf4g1e/_remote_module_non_scriptable.py


/home/yale1/miniconda3/envs/smart-cities/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'loss_function' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_function'])`.


 -- rsync --
Calling: rsync -av /share/temp/yhartmann/smart-cities-journal-based-on-jonahs-ma/data/ /share/data/yhartmann/data/ma-jonah/
sending incremental file list

sent 2,874 bytes  received 31 bytes  5,810.00 bytes/sec
total size is 20,262,785,272  speedup is 6,975,141.23
 -- rsync finished --

Base path: /share/data/yhartmann/data/ma-jonah/


In [8]:
pytorch_to_keras(model, torch.randn(1, *shape), verbose=True)

2024-04-12 13:47:31,358 - pytorch2keras - INFO - Converter is called.
Exported graph: graph(%input_0 : Float(1, 1, 480, 848, strides=[407040, 407040, 848, 1], requires_grad=0, device=cpu),
      %layers.0.weight : Float(8, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cpu),
      %layers.0.bias : Float(8, strides=[1], requires_grad=1, device=cpu),
      %layers.3.weight : Float(16, 8, 3, 3, strides=[72, 9, 3, 1], requires_grad=1, device=cpu),
      %layers.3.bias : Float(16, strides=[1], requires_grad=1, device=cpu),
      %layers.6.weight : Float(32, 16, 3, 3, strides=[144, 9, 3, 1], requires_grad=1, device=cpu),
      %layers.6.bias : Float(32, strides=[1], requires_grad=1, device=cpu),
      %layers.10.weight : Float(4096, 39104, strides=[39104, 1], requires_grad=1, device=cpu),
      %layers.10.bias : Float(4096, strides=[1], requires_grad=1, device=cpu),
      %layers.12.weight : Float(1024, 4096, strides=[4096, 1], requires_grad=1, device=cpu),
      %layers.12.bias : Fl

/home/yale1/miniconda3/envs/smart-cities/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


ValueError: Argument `name` must be a string and cannot contain character `/`. Received: name=/layers/layers.0/Conv_output_0_pad (of type <class 'str'>)

: 